In [7]:
import pandas 

dfs = pandas.read_excel('C2011-07 Patient Transport System.xlsx', sheet_name=None,
    skiprows=1)
sheetnames = list(dfs.keys())

In [35]:
def parse_date(date):
    elms = date.split()
    total_hour = 0
    for elm in elms:
        if elm.endswith("d"):
            total_hour += int(elm[:-1])*24
        elif elm.endswith("h"):
            total_hour += int(elm[:-1])
    return total_hour

def find_column_indices(header, col_names):
    indices = []
    for name in col_names:
        for i, h in enumerate(header):
            if h == name:
                indices.append(i)
    return indices

In [31]:
import os 
if not os.path.isdir("data"):
    os.makedirs("data")
baselines = dfs['Baseline Schedule'][['ID', 'Duration']].values
baselines[:,1] = [parse_date(x) for x in baselines[:,1]]

In [43]:
# planned duration
PD = baselines[0,1]
# tracking periods
tracking_periods = [x for x in sheetnames if "TP" in x]
n_tracking_periods = len(tracking_periods)
print("PD:", PD)
print("Number of tracking periods:", n_tracking_periods)
# init trend
Ts_AT = [PD/n_tracking_periods]
Ts_ES = [PD/n_tracking_periods]
print("T0_AT = T0_ES: ", Ts_AT[0])

PD: 9336
Number of tracking periods: 23
T0_AT = T0_ES:  405.9130434782609


In [56]:
# Col 0 = ID, col 12 = Duration
beta = 0.15
ATs = [0] # init AT0 = 0
t = 1
EVs = [0]
PVs = [0]
ESs = [0]
EACs = [] # predict project duration
for period in tracking_periods:
    print("Tracking periods:", period)
    cols = find_column_indices(dfs[period].values[1], ["ID", "Duration", "Earned Value (EV)", "Planned Value (PV)"])
    data_period = dfs[period].values[2:, cols] 
    data_period[:,1] = [parse_date(x) for x in data_period[:,1]]
    assert (baselines[:,0] == data_period[:,0]).sum() == len(baselines), "Wrong permutation!"

    # current trend
    cur_AT = data_period[0,1]
    ATs.append(cur_AT)
    T_AT = beta*(ATs[t] - ATs[t-1]) + (1-beta)*Ts_AT[t-1]
    Ts_AT.append(T_AT)

    EV = data_period[0,2]
    PV = data_period[0,3]
    PVs.append(PV)
    EVs.append(EV)
    if PVs[t] == PVs[t-1]:
        ES = t-1
    else:
        ES = t-1 + (EV-PVs[t-1]) / (PVs[t]-PVs[t-1])
    ESs.append(ES)
    T_ES = beta*(ESs[t] - ESs[t-1]) + (1-beta)*Ts_ES[t-1]
    Ts_ES.append(T_ES)

    k = (PD-ESs[t]) / T_ES
    EAC = ATs[t] + k * T_AT
    EAC = ATs[t] + (n_tracking_periods-t)*T_AT
    EACs.append(EAC)
    print("EAC:", EAC, k)
    # end calculate
    t += 1

Tracking periods: Project Control - TP1
EAC: 47735.37391304348 27.044167753068695
Tracking periods: TP2
EAC: 40491.85565217391 31.8166679447867
Tracking periods: TP3
EAC: 39008.24347826086 31.77735908131532
Tracking periods: TP4
EAC: 37524.63130434782 31.790190312949782
Tracking periods: TP5
EAC: 36041.01913043478 31.78678373447647
Tracking periods: TP6
EAC: 34557.406956521736 31.802347279319168
Tracking periods: TP7
EAC: 33073.79478260869 31.848681946606575
Tracking periods: TP8
EAC: 31590.18260869565 33.39925173329627
Tracking periods: TP9
EAC: 30106.570434782607 30.501112978152804
Tracking periods: TP10
EAC: 28622.958260869564 32.08137947623256
Tracking periods: TP11
EAC: 27139.34608695652 31.51903211337108
Tracking periods: TP12
EAC: 25655.733913043478 31.943691076638466
Tracking periods: TP13
EAC: 24172.12173913043 31.624025447637408
Tracking periods: TP14
EAC: 22688.50956521739 31.738277705354076
Tracking periods: TP15
EAC: 21204.897391304345 31.783470880346055
Tracking periods: 

In [53]:
data_period

array([[0, 9336, 180759.44, 180759.44],
       [50, 1032, 5512.64, 5512.64],
       [1, 4, 125.12, 125.12],
       [2, 2, 5137.28, 5137.28],
       [3, 24, 250.24, 250.24],
       [51, 24, 2478.88, 2478.88],
       [4, 4, 2416.32, 2416.32],
       [5, 2, 62.56, 62.56],
       [52, 768, 2193.6, 2193.6],
       [6, 48, 942.4, 942.4],
       [7, 48, 500.48, 500.48],
       [8, 72, 750.72, 750.72],
       [53, 528, 3225.6, 3225.6],
       [9, 72, 750.72, 750.72],
       [10, 4, 1974.4, 1974.4],
       [11, 48, 500.48, 500.48],
       [54, 1728, 6334.08, 6334.08],
       [12, 72, 750.72, 750.72],
       [13, 24, 4832.64, 4832.64],
       [14, 24, 250.24, 250.24],
       [15, 48, 500.48, 500.48],
       [55, 1776, 3115.12, 3115.12],
       [16, 72, 750.72, 750.72],
       [17, 4, 1863.92, 1863.92],
       [18, 48, 500.48, 500.48],
       [56, 4, 55373.52, 55373.52],
       [19, 4, 55373.52, 55373.52],
       [57, 288, 3460.8, 3460.8],
       [20, 48, 2710.08, 2710.08],
       [21, 72, 750.72